# Data Science Pandas - Joins, Time Series & Rolling Statistics

## Tasks Today:

1) <b>Pandas</b> <br>
 &nbsp;&nbsp;&nbsp;&nbsp; a) Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - merge() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Breaking Down the Inner Join <br>
 &nbsp;&nbsp;&nbsp;&nbsp; b) Outer Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Use Case Example <br>
 &nbsp;&nbsp;&nbsp;&nbsp; c) Rolling Statistics and Time Series w/Pandas <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - read_csv() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - shift() <br>

In [1]:
import pandas as pd
import numpy as np

### Joins <br>
<p>Very often you will have to combine data from several different sources to obtain the actual dataset you need for your exploration or modeling. Pandas draws heavily on SQL in its API design for joins, and if you are familiar with SQL joins, then Pandas will come naturally. Imagine you have two tables, one with customer name and gender, and the other with their purchases:</p>

In [3]:
# Generate some fake data
city_dict = {1: 'Chicago',
            2: 'San Francisco',
            3: 'New York',
            4: 'Boston',
            5: 'Philadelphia'}
ages = np.random.randint(18, 65, 100)
city = [city_dict[num] for num in np.random.randint(1, 6, 100)]
customer_id = np.arange(1000, 1100)

<p>Let's assume we have a dataset containing the customer ID's cities, and ages for a set of customers. Suppose further, that we have a second table, containing some customer ID's, and a feedback survey rating from thoes customers. Suppose finally, that you as an analyst are asked the following question: What city tends to give the best feedback for our product? We'll need a join operation to get the answer.</p>

In [6]:
customers = pd.DataFrame.from_dict({'ages': ages, 'city': city, 'customer_id': customer_id})
customers.head()

,ages,city,customer_id
0,48,Boston,1000
1,51,Philadelphia,1001
2,42,New York,1002
3,54,Philadelphia,1003
4,45,Boston,1004


In [7]:
# Generate some other fake data
customer_id = np.random.randint(1000, 1200, 100)
customer_rating = np.random.randint(0, 10, 100)

surveys = pd.DataFrame.from_dict({'customer_id': customer_id, 'customer_rating': customer_rating})
surveys.head()

,customer_id,customer_rating
0,1124,7
1,1074,7
2,1118,0
3,1101,9
4,1126,9


##### merge()

<p>We can zip the two dataframes up with an inner join, using the Pandas API as follows:</p>

In [8]:
customer_merge = customers.merge(surveys, on="customer_id", how='inner')
customer_merge.head()

,ages,city,customer_id,customer_rating
0,54,Philadelphia,1003,6
1,54,Philadelphia,1003,4
2,20,San Francisco,1008,3
3,48,Boston,1009,2
4,24,Boston,1010,0


<p>Now it's a simple groupby operation to reveal the answer:</p>

In [17]:
customer_merge.groupby('city').mean().drop(['ages', 'customer_id'], axis=1)

,customer_rating
city,
Boston,3.444444
Chicago,4.000000
New York,4.333333
Philadelphia,4.533333
San Francisco,4.750000


##### Breaking Down the Inner Join <br>
<p>Let's take a closer look at the actual mechanics here however. There is some subtlety and nuance that is best understood by thinking of an inner join as two steps: a cartesian product, and then a filter. Observe the result of the following inner join:</p>

<p>Somehow, we ended up with more rows in the result than either of the original tables! This is because we joined on a $\textit{non-unique index}$. The join starts by computing all the possible combinations of rows, and then filters them based on the condition. Because of the duplicates, multiple combinations of rows make it through!</p>

### Outer Joins <br>
<p>An $\textit{outer join}$ produces all of the rows from the left table, and joins whatever rows it can to the right table, filling anywhere it cant with NaN's or None values. Below is a brief example, and then we will discuss a possible use case for this type of join.</p>

In [18]:
df1 = pd.DataFrame.from_dict({'A': [1,2,3], 'B': ['a','b','c']})
df2 = pd.DataFrame.from_dict({'A': [1], 'C': ['d']})

In [19]:
df1.head()

,A,B
0,1,a
1,2,b
2,3,c


In [20]:
df2

,A,C
0,1,d


In [22]:
df1.merge(df2, on='A', how='outer')

,A,B,C
0,1,a,d
1,2,b,NaN
2,3,c,NaN


##### Use Case Example

<p>As an example use case, let's imagine we have again the demographics and survey response tables from above:</p>

In [23]:
customers.head()

,ages,city,customer_id
0,48,Boston,1000
1,51,Philadelphia,1001
2,42,New York,1002
3,54,Philadelphia,1003
4,45,Boston,1004


In [24]:
surveys.head()

,customer_id,customer_rating
0,1124,7
1,1074,7
2,1118,0
3,1101,9
4,1126,9


<p>Now, you are asked as an analyst to compare the demographics of survey responders to the demographics of all your customers combined.</p>

In [26]:
# outer join 
customer_outer = customers.merge(surveys, on='customer_id', how='outer')
customer_outer.head()

,ages,city,customer_id,customer_rating
0,48.0,Boston,1000,NaN
1,51.0,Philadelphia,1001,NaN
2,42.0,New York,1002,NaN
3,54.0,Philadelphia,1003,6.0
4,54.0,Philadelphia,1003,4.0


In [36]:
customer_no_rating = customer_outer[pd.isna(customer_outer['customer_rating'])]
customer_no_rating.shape

(61, 4)

In [43]:
customer_outer['customer_rating'].value_counts(dropna = False)

NaN     61
 7.0    13
 1.0    13
 0.0    13
 8.0    12
 9.0    10
 2.0    10
 3.0     8
 4.0     8
 6.0     8
 5.0     5
Name: customer_rating, dtype: int64

In [42]:
customer_merge.groupby(['city', 'customer_rating']).size()

city           customer_rating
Boston         0                  2
               1                  1
               2                  2
               4                  1
               5                  1
               8                  1
               9                  1
Chicago        0                  1
               1                  1
               2                  2
               6                  2
               7                  1
               8                  1
New York       0                  1
               3                  2
               6                  2
               8                  1
Philadelphia   0                  1
               1                  3
               3                  2
               4                  2
               5                  1
               6                  1
               7                  2
               8                  1
               9                  2
San Francisco  0                 

In [47]:
customer_no_rating_age = customer_outer.loc[pd.isna(customer_outer['customer_rating']), 'ages']
customer_no_rating_age.mean()

40.01639344262295

In [49]:
customer_has_rating_age = customer_outer.loc[pd.notna(customer_outer['customer_rating']), 'ages']
customer_has_rating_age.mean()

45.34

### Rolling Statistics and Time Series with Pandas

##### read_csv()

In [50]:
weather = pd.read_csv('chiweather.csv', sep=';')
weather.head()

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd]
0,2017,11,25,0,0,51.85,0.0,17.31,327.13
1,2017,11,25,1,0,49.14,0.0,20.53,330.64
2,2017,11,25,2,0,46.42,0.0,21.41,327.80
3,2017,11,25,3,0,44.13,0.0,21.79,324.19
4,2017,11,25,4,0,42.22,0.0,21.03,321.91


In [51]:
weather = weather.groupby(['Year', 'Month', 'Day']).mean().reset_index()
weather

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd]
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000
5,2017,11,30,11.5,0.0,43.618750,0.033333,14.343333,276.060000
6,2017,12,1,11.5,0.0,41.860417,0.000000,6.845000,223.459583
7,2017,12,2,11.5,0.0,45.324167,0.000000,8.934167,218.252500


##### shift()

In [5]:
# Shift index by desired number of periods with an optional time freq

In [54]:
weather['last_temp'] = weather['Temperature  [2 m above gnd]'].shift(1)
weather

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd],last_temp
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667,NaN
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750,41.208750
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083,38.298333
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333,45.611250
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000,48.497500
5,2017,11,30,11.5,0.0,43.618750,0.033333,14.343333,276.060000,40.122917
6,2017,12,1,11.5,0.0,41.860417,0.000000,6.845000,223.459583,43.618750
7,2017,12,2,11.5,0.0,45.324167,0.000000,8.934167,218.252500,41.860417
